In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN
from keras.models import Sequential
from IPython.display import Image, display
print("Using tensorflow:",tf.__version__)

Using tensorflow: 2.4.1


In [2]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project='Assignment 3', entity='iitm-cs6910-jan-may-2021-cs20m059-cs20m007')
wandb.run.name = "LSTM-Transliteration-" + datetime.datetime.now().isoformat()
wandb.run.save()

In [3]:
val_df = pd.read_csv("./lexicons/hi.translit.sampled.dev.tsv", sep='\t', header=None)
train_df = pd.read_csv("./lexicons/hi.translit.sampled.train.tsv", sep='\t', header=None)
test_df = pd.read_csv("./lexicons/hi.translit.sampled.test.tsv", sep='\t', header=None)
print("Data Loaded to Dataframes!")

Data Loaded to Dataframes!


In [4]:
# Process the dataframe to 
def create_dataset(data_frame):
    input_words = []
    target_words = []
    for x, y in zip(data_frame[1], data_frame[0]):
        # Add words to respective lists
        input_words.append("@"+str(x)+"#")
        target_words.append("@"+str(y)+"#")
    return input_words, target_words

In [5]:
def tokenize(words):
    # Initialize the tokenizer
    tokenizer = Tokenizer(num_words=None, char_level=True)
    
    # Fit on the set of words
    tokenizer.fit_on_texts(words)
    tensor = tokenizer.texts_to_sequences(words)
    
    #Pad the smaller words
    tensor = pad_sequences(tensor, padding='post')
    
    # Return the tensor and the tokenizer
    return tensor, tokenizer

In [6]:
def load_dataset(data_frame):
    input_words, target_words = create_dataset(data_frame)

    input_tensor, inp_word_tokenizer = tokenize(input_words)
    target_tensor, targ_word_tokenizer = tokenize(target_words)

    return input_tensor, target_tensor, inp_word_tokenizer, targ_word_tokenizer

In [22]:
val_input_tensor, val_target_tensor, val_inp_tk, val_targ_tk = load_dataset(val_df)
train_input_tensor, train_target_tensor, train_inp_tk, train_targ_tk = load_dataset(train_df)
test_input_tensor, test_target_tensor, test_inp_tk, test_targ_tk = load_dataset(test_df)

print(f'Shape of Val input tensor: {np.shape(val_input_tensor)} | Shape of Val target tensor: {np.shape(val_target_tensor)}')
print(f'Shape of Train input tensor: {np.shape(train_input_tensor)} | Shape of Train target tensor: {np.shape(train_target_tensor)}')
print(f'Shape of Test input tensor: {np.shape(test_input_tensor)} | Shape of Test target tensor: {np.shape(test_target_tensor)}')

Shape of Val input tensor: (4358, 20) | Shape of Val target tensor: (4358, 16)
Shape of Train input tensor: (44204, 22) | Shape of Train target tensor: (44204, 21)
Shape of Test input tensor: (4502, 18) | Shape of Test target tensor: (4502, 17)


In [26]:
def convert(tk, tensor):
    for t in tensor:
        if t != 0:
            print(f'{t} ----> {tk.index_word[t]}')